In [9]:
import pandas as pd
df = pd.read_csv('../data/clean.csv')
df.rename(columns={"SName": "title", "Lyric": "ctext"},inplace = True)
df

,title,ctext,Number_of_words,Number_of_words_title
0,careless whisper,i feel so unsure as i take your hand and lead ...,299,2
1,could you be loved,"don't let them fool, ya or even try to school,...",161,11
2,cruisin',"baby, let's cruise, away from here don't be co...",159,3
3,easy,"know it sounds funny but, i just can't stand t...",175,1
4,for your babies,you've got that look again the one i hoped i h...,215,6
...,...,...,...,...
191809,the waiting,chorus here we stand waiting on the plain dark...,198,2
191810,too early for the sky,i nearly disappeared into the mouth of a croco...,137,5
191811,warsaw 1943,"amambuka, amambuka azothengisa izwe lakithi, i...",221,7
191812,when the system has fallen,sweat in the heat for days on end waiting for ...,260,5


In [2]:
# df.to_csv('./data/renamed_clean.csv',index=False)

# Fine Tuning Transformer for Summary Generation (Song title)


### Introduction

In this tutorial we will be fine tuning a transformer model for **Summarization Task**. 
In this task a song title of a given lyric is generated when passed through a network. There are 2 types of summary generation mechanisms:

1. ***Extractive Summary:*** the network calculates the most important sentences from the text and gets them together to provide the most meaningful information from the text.
2. ***Abstractive Summary***: The network creates new sentences to encapsulate maximum gist of the text and generates that as output. The sentences in the summary may or may not be contained in the text. 

In this tutorial we will be generating ***Abstractive Summary***. 

#### Flow of the notebook

* In the tutorial, most of the sections have been created into functions, and they are called from the `main()` in the end of the notebook. 
* This is done to leverage the [Weights and Biases Service](https://www.wandb.com/) WandB in short.
* It is a experiment tracking, parameter optimization and artifact management service. That can be very easily integrated to any of the Deep learning or Machine learning frameworks. 

The notebook will be divided into separate sections to provide a organized walk through for the process used. This process can be modified for individual use cases. The sections are:

1. [Preparing Environment and Importing Libraries](#section01)
2. [Preparing the Dataset for data processing: Class](#section02)
3. [Fine Tuning the Model: Function](#section03)
4. [Validating the Model Performance: Function](#section04)
5. [Main Function](#section05)
    * [Initializing WandB](#section501)
    * [Importing and Pre-Processing the domain data](#section502)
    * [Creation of Dataset and Dataloader](#section503)
    * [Neural Network and Optimizer](#section504)
    * [Training Model and Logging to WandB](#section505)
    * [Validation and generation of Summary](#section506)
6. [Examples of the Summary Generated from the model](#section06)


#### Technical Details

This script leverages on multiple tools designed by other teams. Details of the tools used below. Please ensure that these elements are present in your setup to successfully implement this script.

- **Data**:
	- We are using the Song lyrics from 79 musical genres dataset available at [Kaggle](https://www.kaggle.com/datasets/neisse/scrapped-lyrics-from-6-genres?select=lyrics-data.csv)
	- All the data were obtained by scraping the Brazilian website Vagalume using R by the ower of the data set on kaggle. We are referring only to the second csv file from the data dump: `lyrics-data.csv`
	- After preprocessing, there are`191814` rows of data.  Where each row has the following data-point:
		- **title** : Song's name
		- **ctext** : Song lyrics
		- **Number_of_words**: Number of words in lyrics
		- **Number_of_words_title**: Number of words in title


- **Language Model Used**: 
    - This notebook uses one of the most recent and novel transformers model ***T5***. [Research Paper](https://arxiv.org/abs/1910.10683)    
    - ***T5*** in many ways is one of its kind transformers architecture that not only gives state of the art results in many NLP tasks, but also has a very radical approach to NLP tasks.
    - **Text-2-Text** - According to the graphic taken from the T5 paper. All NLP tasks are converted to a **text-to-text** problem. Tasks such as translation, classification, summarization and question answering, all of them are treated as a text-to-text conversion problem, rather than seen as separate unique problem statements.
    - **Unified approach for NLP Deep Learning** - Since the task is reflected purely in the text input and output, you can use the same model, objective, training procedure, and decoding process to ANY task. Above framework can be used for any task - show Q&A, summarization, etc. 
   - We will be taking inputs from the T5 paper to prepare our dataset prior to fine tuning and training.    
   - [Documentation for python](https://huggingface.co/transformers/model_doc/t5.html)

![**Each NLP problem as a “text-to-text” problem** - input: text, output: text](https://miro.medium.com/max/4006/1*D0J1gNQf8vrrUpKeyD8wPA.png) 
	 


- Hardware Requirements: 
	- Python 3.6 and above
	- Pytorch, Transformers and
	- All the stock Python ML Library
	- GPU enabled setup 
   

- **Script Objective**:
	- The objective of this script is to fine tune ***T5 *** to be able to generate title (summary task), that a close to or better than the actual title while ensuring the important information from the lyrics is not lost.

---
NOTE: 
We are using the Weights and Biases Tool-set in  this tutorial. The different components will be explained as we go through the article.

<a id='section01'></a>
### Preparing Environment and Importing Libraries

At this step we will be installing the necessary libraries followed by importing the libraries and modules needed to run our script. 
We will be installing:
* transformers
* wandb

Libraries imported are:
* Pandas
* Pytorch
* Pytorch Utils for Dataset and Dataloader
* Transformers
* T5 Model and Tokenizer
* wandb

Followed by that we will preapre the device for CUDA execeution. This configuration is needed if you want to leverage on onboard GPU. First we will check the GPU avaiable to us, using the nvidia command followed by defining our device.

Finally, we will be logging into the [wandb](https://www.wandb.com/) serice using the login command

In [3]:
!pip install transformers -q
!pip install wandb -q

# Code for TPU packages install
# !curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

In [1]:
# Importing stock libraries
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

# WandB – Import the wandb library
import wandb

In [5]:
# Checking out the GPU we have access to. This is output is from the google colab version. 
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [6]:
# # Setting up the device for GPU usage
from torch import cuda
device = torch.device(4) if cuda.is_available() else torch.device('cpu')
device
# Preparing for TPU usage
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

device(type='cpu')

In [7]:
# Login to wandb to log the model run and all the parameters
!wandb login #0bb1fe48885469a7c6ad87a0d4e7dbbf5ef052b3

wandb: Appending key for api.wandb.ai to your netrc file: /Users/fanli/.netrc


<a id='section02'></a>
### Preparing the Dataset for data processing: Class

We will start with creation of Dataset class - This defines how the text is pre-processed before sending it to the neural network. This dataset will be used the the Dataloader method that will feed  the data in batches to the neural network for suitable training and processing. 
The Dataloader and Dataset will be used inside the `main()`.
Dataset and Dataloader are constructs of the PyTorch library for defining and controlling the data pre-processing and its passage to neural network. For further reading into Dataset and Dataloader read the [docs at PyTorch](https://pytorch.org/docs/stable/data.html)

#### *CustomDataset* Dataset Class
- This class is defined to accept the Dataframe as input and generate tokenized output that is used by the **T5** model for training. 
- We are using the **T5** tokenizer to tokenize the data in the `text` and `ctext` column of the dataframe. 
- The tokenizer uses the ` batch_encode_plus` method to perform tokenization and generate the necessary outputs, namely: `source_id`, `source_mask` from the actual lyric text and `target_id` and `target_mask` from the title text.
- To read further into the tokenizer, [refer to this document](https://huggingface.co/transformers/model_doc/t5.html#t5tokenizer)
- The *CustomDataset* class is used to create 2 datasets, for training and for validation.
- *Training Dataset* is used to fine tune the model: **80% of the original data**
- *Validation Dataset* is used to evaluate the performance of the model. The model has not seen this data during training. 

#### Dataloader: Called inside the `main()`
- Dataloader is used to for creating training and validation dataloader that load data to the neural network in a defined manner. This is needed because all the data from the dataset cannot be loaded to the memory at once, hence the amount of data loaded to the memory and then passed to the neural network needs to be controlled.
- This control is achieved using the parameters such as `batch_size` and `max_len`.
- Training and Validation dataloaders are used in the training and validation part of the flow respectively

In [8]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage for finetuning the model and to prepare it for predictions

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.title
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

<a id='section03'></a>
### Fine Tuning the Model: Function

Here we define a training function that trains the model on the training dataset created above, specified number of times (EPOCH), An epoch defines how many times the complete data will be passed through the network. 

This function is called in the `main()`

Following events happen in this function to fine tune the neural network:
- The epoch, tokenizer, model, device details, testing_ dataloader and optimizer are passed to the `train ()` when its called from the `main()`
- The dataloader passes data to the model based on the batch size.
- `language_model_labels` are calculated from the `target_ids` also, `source_id` and `attention_mask` are extracted.
- The model outputs first element gives the loss for the forward pass. 
- Loss value is used to optimize the weights of the neurons in the network.
- After every 10 steps the loss value is logged in the wandb service. This log is then used to generate graphs for analysis. Such as [these](https://wandb.ai/victorlifan/transformers_tutorials_summarization?workspace=user-victorlifan)
- After every 500 steps the loss value is printed in the console.

In [9]:
# Creating the training function. This will be called in the main function. It is run depending on the epoch value.
# The model is put into train mode and then we enumerate over the training loader and passed to the defined network 

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
        loss = outputs[0]
        
        if _%10 == 0:
            wandb.log({"Training Loss": loss.item()})

        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

<a id='section04'></a>
### Validating the Model Performance: Function

During the validation stage we pass the unseen data(Testing Dataset), trained model, tokenizer and device details to the function to perform the validation run. This step generates new summary for dataset that it has not seen during the training session. 

This function is called in the `main()`

This unseen data is the 20% of `lyrics-data.csv` which was seperated during the Dataset creation stage. 
During the validation stage the weights of the model are not updated. We use the generate method for generating new text for the summary. 

It depends on the `Beam-Search coding` method developed for sequence generation for models with LM head. 

The generated titles and originally titles are decoded from tokens to text and returned to the `main()`

[text_generation parameters](https://huggingface.co/docs/transformers/v4.19.2/en/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate)

In [ ]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=24, 
                min_length = 1,
                num_beams=5,
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

<a id='section05'></a>
### Main Function

The `main()` as the name suggests is the central location to execute all the functions/flows created above in the notebook. The following steps are executed in the `main()`:


<a id='section501'></a>
#### Initializing WandB 

* The `main()` begins with initializing WandB run under a specific project. This command initiates a new run for each execution of this command. 

* Before we proceed any further i will give a brief overview of the **[WandB Service](https://www.wandb.com/)**

* This service has been created to track ML experiments, Optimize the experiments and save artifacts. It is designed to seamlessly integrate with all the Machine Learning and Deep Learning Frameworks. Each script can be organized into *Project* and each execution of the script will be registered as a *run* in the respective project.

* The service can be configured to log several default metrics, such a network weights, hardware usage, gradients and weights of the network. 

* It can also be used to log user defined metrics, such a loss in the `train()`.

* This particular tutorial is logged in the project: **[transformers_tutorials_summarization](https://wandb.ai/victorlifan/transformers_tutorials_summarization)**

**One of the charts from the project**
![](meta/wandb.png)

* Visit the project page to see the details of different runs and what information is logged by the service. 

* Following the initialization of the WandB service we define configuration parameters that will be used across the tutorial such as `batch_size`, `epoch`, `learning_rate` etc.

* These parameters are also passed to the WandB config. The config construct with all the parameters can be optimized using the Sweep service from WandB. Currently, that is outof scope of this tutorial. 

* Next we defining seed values so that the experiment and results can be reproduced.


<a id='section502'></a>
#### Importing and Pre-Processing the domain data

We will be working with the data and preparing it for fine tuning purposes. 
*Assuming that the `lyrics-data.csv` is already downloaded in your `data` folder*

* The file is imported as a dataframe and give it the headers as per the documentation.
* Cleaning the file to remove the unwanted columns.
* A new string is added to the main article column `summarize: ` prior to the actual article. This is done because **T5** had similar formatting for the summarization dataset. 
* The final Dataframe will be something like this:

|title|ctext|
|--|--|
|title-1|summarize: lyric 1|
|title-2|summarize: lyric 2|
|title-3|summarize: lyric 3|

* Top 5 rows of the dataframe are printed on the console.

<a id='section503'></a>
#### Creation of Dataset and Dataloader

* The updated dataframe is divided into 80-20 ratio for test and validation. 
* Both the data-frames are passed to the `CustomerDataset` class for tokenization of the new articles and their summaries.
* The tokenization is done using the length parameters passed to the class.
* Train and Validation parameters are defined and passed to the `pytorch Dataloader contstruct` to create `train` and `validation` data loaders.
* These dataloaders will be passed to `train()` and `validate()` respectively for training and validation action.
* The shape of datasets is printed in the console.


<a id='section504'></a>
#### Neural Network and Optimizer

* In this stage we define the model and optimizer that will be used for training and to update the weights of the network. 
* We are using the `t5-base` transformer model for our project. You can read about the `T5 model` and its features above. 
* We use the `T5ForConditionalGeneration.from_pretrained("t5-base")` commad to define our model. The `T5ForConditionalGeneration` adds a Language Model head to our `T5 model`. The Language Model head allows us to generate text based on the training of `T5 model`.
* We are using the `Adam` optimizer for our project. This has been a standard for all our tutorials and is something that can be changed updated to see how different optimizer perform with different learning rates. 
* There is also a scope for doing more with Optimizer such a decay, momentum to dynamically update the Learning rate and other parameters. All those concepts have been kept out of scope for these tutorials. 


<a id='section505'></a>
#### Training Model and Logging to WandB

* Now we log all the metrics in WandB project that we have initialized above.
* Followed by that we call the `train()` with all the necessary parameters.
* Loss at every 500th step is printed on the console.
* Loss at every 10th step is logged as Loss in the WandB service.


<a id='section506'></a>
#### Validation and generation of Summary

* After the training is completed, the validation step is initiated.
* As defined in the validation function, the model weights are not updated. We use the fine tuned model to generate new titles based on the song lyric.
* An output is printed on the console giving a count of how many steps are complete after every 100th step. 
* The original title and generated title are converted into a list and returned to the main function. 
* Both the lists are used to create the final dataframe with 2 columns **Generated Title** and **Actual Title**
* The dataframe is saved as a csv file in the local drive.
* A qualitative analysis can be done with the Dataframe. 

In [11]:
def main():
    # WandB – Initialize a new run
    wandb.init(project="transformers_tutorials_summarization")

    # WandB – Config is a variable that holds and saves hyperparameters and inputs
    # Defining some key variables that will be used later on in the training  
    config = wandb.config          # Initialize config
    config.TRAIN_BATCH_SIZE = 16   # input batch size for training (default: 64)
    config.VALID_BATCH_SIZE = 16    # input batch size for testing (default: 1000)
    config.TRAIN_EPOCHS = 10        # number of epochs to train (default: 10)
    config.VAL_EPOCHS = 1 
    config.LEARNING_RATE = 1e-5    # learning rate (default: 0.01)
    config.SEED = 42               # random seed (default: 42)
    config.MAX_LEN = 512
    config.SUMMARY_LEN = 24 

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(config.SEED) # pytorch random seed
    np.random.seed(config.SEED) # numpy random seed
    torch.backends.cudnn.deterministic = True

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    

    # Importing and Pre-Processing the domain data
    # Selecting the needed columns only. 
    # Adding the 'summarzie' text in front of the lyric. This is to format the dataset similar to how T5 model was trained for summarization task. 
    df = pd.read_csv('../data/renamed_clean.csv',encoding='latin-1')
    df = df[['title','ctext']]
    df.ctext = 'summarize: ' + df.ctext
    print(df.head())

    
    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest will be used for validation. 
    train_size = 0.8
    train_dataset=df.sample(frac=train_size,random_state = config.SEED)
    val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    print("FULL Dataset: {}".format(df.shape))
    print("TRAIN Dataset: {}".format(train_dataset.shape))
    print("TEST Dataset: {}".format(val_dataset.shape))


    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
    val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

    # Defining the parameters for creation of dataloaders
    train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

    val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)


    
    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
    model = model.to(device)

    # Defining the optimizer that will be used to tune the weights of the network in the training session. 
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

    # Log metrics with wandb
    wandb.watch(model, log="all")
    # Training loop
    print('Initiating Fine-Tuning for the model on our dataset')

    for epoch in range(config.TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)


    # Validation loop and saving the resulting file with predictions and acutals in a dataframe.
    # Saving the dataframe as predictions.csv
    print('Now generating titles on our fine tuned model for the validation dataset and saving it in a dataframe')
    for epoch in range(config.VAL_EPOCHS):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Generated Title':predictions,'Actual Title':actuals})
        final_df.to_csv('notebook_predictions.csv')
        print('Output Files generated for review')

if __name__ == '__main__':
    main()

wandb: Currently logged in as: victorlifan. Use `wandb login --relogin` to force relogin


                title                                              ctext
0    careless whisper  summarize: i feel so unsure as i take your han...
1  could you be loved  summarize: don't let them fool, ya or even try...
2            cruisin'  summarize: baby, let's cruise, away from here ...
3                easy  summarize: know it sounds funny but, i just ca...
4     for your babies  summarize: you've got that look again the one ...
FULL Dataset: (191814, 2)
TRAIN Dataset: (153451, 2)
TEST Dataset: (38363, 2)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Initiating Fine-Tuning for the model on our dataset


/Users/fanli/opt/anaconda3/envs/bc/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2271: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Loss:  6.86549711227417


KeyboardInterrupt: 

<a id='section06'></a>
### Examples of the Titles Generated from the model
### Three versions of model performance comparation

In [33]:
result_df = pd.read_csv('../results/predictions.csv',index_col=0)
result_df2 = pd.read_csv('../results/V2_predictions.csv',index_col=0)
result_df3 = pd.read_csv('../results/V3_predictions.csv',index_col=0)
# randomly select 20 songs from validation set
idx = np.random.randint(0,high = len(result_df),size=20)

In [34]:
result_df = result_df.iloc[idx].rename({"Generated Title":"V1_Generated Title"},axis = 1)
result_df2 = result_df2.iloc[idx].rename({"Generated Title":"V2_Generated Title"},axis = 1)['V2_Generated Title']
result_df3 = result_df3.iloc[idx].rename({"Generated Title":"V3_Generated Title"},axis = 1)['V3_Generated Title']

In [37]:
comb_df = result_df.join(result_df2).join(result_df3).iloc[:,[0,2,3,1]]
comb_df

,V1_Generated Title,V2_Generated Title,V3_Generated Title,Actual Title
37876,if there's a place where,- findaway time,don't know how to feel,another time
22376,dream lover,dream lover,another dream lover,dream lover
31055,beethoven,beethoven,beethoven,roll over beethoven
12386,NaN,if i try,bleed,breathe no more
2774,NaN,- moon,azure,moon song
5437,a lady like you,a lady like you,a lady like you,a lady like you
25564,- apart,- apart,we go together,together
5325,hermade him wrong,hermade him wrong,she put the sad in all his songs,she put the sad in all his songs
28928,NaN,thing i need,run away with me,runaway
26458,NaN,& i,zig-zag,lightning


## Comparing with huggingface autotrain model: `autotrain-song_title_generate-939531516`

In [30]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def generate(lyric):
        
    tokenizer = AutoTokenizer.from_pretrained("victorlifan/autotrain-song_title_generate-939531516")
    model = AutoModelForSeq2SeqLM.from_pretrained("victorlifan/autotrain-song_title_generate-939531516")
    
    inputs = tokenizer.encode("summarize: " + lyric, return_tensors="pt")
    outputs = model.generate(inputs, max_length=24, min_length=1, length_penalty=2.0, num_beams=4, early_stopping=True)
    result = tokenizer.decode(outputs[0])
    return result

# inference on valid set
for i in result_df3['Actual Title'].iloc[idx]:
    
    lyric = df[df['title']==i]['ctext'].values[0]
    title = generate(lyric)
    
    #print(f"song id: {idx}\n")
    print(f"actual title: {i}\n")
    print(f"generated title: {title}\n")
    #print(f"lyrics: {df['Lyric'][idx]}\n")
    print('='*50)

Token indices sequence length is longer than the specified maximum sequence length for this model (1352 > 512). Running this sequence through the model will result in indexing errors


actual title: mind of a souljah

generated title: <pad> mind of a soul</s>

actual title: secret lives

generated title: <pad> secret lives</s>

actual title: in cold blood

generated title: <pad> cold blood</s>

actual title: the comet song

generated title: <pad> the comet song</s>



Token indices sequence length is longer than the specified maximum sequence length for this model (722 > 512). Running this sequence through the model will result in indexing errors


actual title: full of fire

generated title: <pad> full of fire</s>

actual title: candy perfume girl

generated title: <pad> candy perfume girl</s>

actual title: ain't no way

generated title: <pad> ain't no way</s>



Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors


...

actual title: it'll be okay

generated title: <pad> it'll be okay</s>

actual title: hard believer

generated title: <pad> love is tough</s>

actual title: the exorcist

generated title: <pad> exorcist</s>

actual title: welcome to paradise

generated title: <pad> welcome to paradise</s>

actual title: nails

generated title: <pad> cutting the ties</s>

actual title: shake yours

generated title: <pad> blues for you</s>



Token indices sequence length is longer than the specified maximum sequence length for this model (600 > 512). Running this sequence through the model will result in indexing errors


actual title: land of fear

generated title: <pad> land of fear</s>

actual title: are you afraid?

generated title: <pad> are you afraid?</s>



Token indices sequence length is longer than the specified maximum sequence length for this model (1116 > 512). Running this sequence through the model will result in indexing errors


actual title: it's alright

generated title: <pad> it's alright</s>

actual title: person of interest

generated title: <pad> person of interest</s>

actual title: the real world

generated title: <pad> too many lonely hearts</s>



# Reference
[PyTorch Transformers Tutorials](https://github.com/abhimishra91/transformers-tutorials)